## Prepping features dataset


Things to look at:
1. We are currently dropping NA in Demand, Temp. and Humidity. --> Temp and humidity is given each 5 minutes, fill up temp. and humidity with average. 
2. Normalization
3. Demand values we have:
12/31/2018  21:15:00  upandincluding 10/11/2021/ 6:07:00
4. Predicting the demand from 
10/11/2021/ 6:08 upandincluding 12/13/2021  17:59:00 

In [51]:
import math, datetime, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import itertools
import tqdm

def load_data(data_path):
  data = pd.read_csv(data_path)  
  return data

inbound = load_data("inbound_loads.csv")
outbound = load_data("outbound_laods.csv")
weather = load_data("weather.csv")
#For loop to ensure that all pallet data is in the same dataframe
pallet = load_data("Pallet_history_Gold_Spike[0].csv")
for x in range(1, 10):
    pallet = pd.concat([pallet, load_data(f"Pallet_history_Gold_Spike[{x}].csv")])
trainentest = load_data("demand_kWtrain_val.csv")
train = trainentest.iloc[:273988,:]
test = trainentest.iloc[273988:, :]

pallet = pallet.drop(['lot_code', 
                      'tran_type', 
                      'final_pallet_code', 
                      'warehouse_facility_id',
                      'source_system_id'], axis=1)

In [16]:
def addtimecol(df, colname): ####input df and colname 
    df[colname] = pd.to_datetime(df[colname])         
    df['year'] = df[colname].dt.year
    df['month'] = df[colname].dt.month
    df['weekday'] = df[colname].dt.weekday
    df['day'] = df[colname].dt.day
    df['hour'] = df[colname].dt.hour
    df['minute'] = df[colname].dt.minute        
    return df
def weekend(x):
    if x['weekday'] > 4:
        return True
    return False
    

In [52]:
trainentest = load_data("demand_kWtrain_val.csv")
train = trainentest.iloc[:273988,:]
train

,Unnamed: 0,datetime_local,demand_kW
0,0,12/31/18 21:15,2064.101392
1,1,12/31/18 21:30,1874.002081
2,2,12/31/18 21:45,1988.168511
3,3,12/31/18 22:00,2022.795943
4,4,12/31/18 22:15,1986.981872
5,5,12/31/18 22:30,1921.142106
6,6,12/31/18 22:45,1956.130266
7,7,12/31/18 23:00,2039.546964
8,8,12/31/18 23:15,2100.701840
9,9,12/31/18 23:30,1901.260583


In [150]:
###Incoming weight feature preprocessing
#load all data
incoming_weight_1h = load_data('incoming_weight_df_1h.csv')
incoming_weight_5h = load_data('incoming_weight_df_5h.csv')
incoming_weight_10h = load_data('incoming_weight_df_10h.csv')
incoming_weight_23h = load_data('incoming_weight_df.csv')
#rename columns 
incoming_weight_5h = incoming_weight_5h.rename(columns = {'weight_23h' : 'weight_5h'})
incoming_weight_1h = incoming_weight_1h.rename(columns = {'weight_23h' : 'weight_1h'})
incoming_weight_10h = incoming_weight_10h.rename(columns = {'weight_23h' : 'weight_10h'})
#get them to a datetime object
incoming_weight_1h['datetime_local'] = pd.to_datetime(incoming_weight_1h['datetime_local'])
incoming_weight_5h['datetime_local'] = pd.to_datetime(incoming_weight_5h['datetime_local'])
incoming_weight_10h['datetime_local'] = pd.to_datetime(incoming_weight_10h['datetime_local'])
incoming_weight_23h['datetime_local'] = pd.to_datetime(incoming_weight_23h['datetime_local'])
#set index to be datetime
incoming_weight_1h.set_index('datetime_local', inplace=True)
incoming_weight_5h.set_index('datetime_local', inplace=True)
incoming_weight_10h.set_index('datetime_local', inplace=True)
incoming_weight_23h.set_index('datetime_local', inplace=True)
#reshape them to start at 2018-31-12 9:15PM
incoming_weight_1h = incoming_weight_1h[2361:]
incoming_weight_5h = incoming_weight_5h[2330:] 
incoming_weight_10h = incoming_weight_10h[2326:]
incoming_weight_23h = incoming_weight_23h[2323:]
#Drop duplicates
incoming_weight_1h = incoming_weight_1h.loc[~incoming_weight_1h.index.duplicated()]
incoming_weight_5h = incoming_weight_5h.loc[~incoming_weight_5h.index.duplicated()]
incoming_weight_10h = incoming_weight_10h.loc[~incoming_weight_10h.index.duplicated()]
incoming_weight_23h = incoming_weight_23h.loc[~incoming_weight_23h.index.duplicated()]


Prepare the pallet movement feature

In [223]:
#load data
pallet_move_5min = load_data('pallet_movement_5min_ft.csv')
#rename column
pallet_move_5min.rename(columns = {'quantity' : 'pallet_movement_5min'}, inplace = True)
#make index a datetime object and set as index
pallet_move_5min['datetime_local'] = pd.to_datetime(pallet_move_5min['datetime_local'])
pallet_move_5min.set_index('datetime_local', inplace = True)
#delete duplicates
pallet_move_5min = pallet_move_5min.loc[~pallet_move_5min.index.duplicated()]

,pallet_movement_5min
datetime_local,
2019-01-02 00:30:00,63
2019-01-02 00:31:00,63
2019-01-02 00:32:00,126
2019-01-02 00:33:00,126
2019-01-02 00:34:00,189
2019-01-02 00:35:00,189
2019-01-02 00:36:00,189
2019-01-02 00:37:00,189
2019-01-02 00:38:00,252


,Unnamed: 0,demand_kW,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC,hour,weight_1h,weight_5h,weight_10h,weight_23h
datetime_local,,,,,,,,,,,,,,,,,
2019-01-05 00:33:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,181756.0,318653.0,1263072.0
2019-01-05 00:34:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,181756.0,318653.0,1263072.0
2019-01-05 00:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-05 00:35:00-06:00,70.06,41.00,2019-01-05 06:35:00,0.0,0.0,181756.0,318653.0,1263072.0
2019-01-05 00:36:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,181756.0,318653.0,1263072.0
2019-01-05 00:37:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,181756.0,318653.0,1263072.0
2019-01-05 00:38:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,181756.0,318653.0,1263072.0
2019-01-05 00:39:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,181756.0,318653.0,1263072.0
2019-01-05 00:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-05 00:40:00-06:00,75.14,39.20,2019-01-05 06:40:00,0.0,0.0,181756.0,318653.0,1263072.0
2019-01-05 00:41:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,144267.0,318653.0,1263072.0


In [225]:
#Create new dummy dfs
base_df = train.copy()
base_weather = weather.copy()

#Remove unnecessary columns
#base_df = base_df.drop('Unnamed: 0', axis=1)
base_weather = base_weather.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
base_weather["localstrptime"]= pd.to_datetime(base_weather["localstrptime"])
base_df['datetime_local'] = pd.to_datetime(base_df['datetime_local'])
base_weather = base_weather.rename(columns={'localstrptime':'datetime_local'})
#base_weather['datetime_local'] = [datetime.datetime.strptime(x, 'yyyy/MM/dd HH:mm:SS') for x in base_weather['localstrptime']]

#Add pallet movements feature
##

#Add time columns.
addtimecol(base_df, 'datetime_local')

#Set index to datetime
base_df.set_index('datetime_local', inplace=True)
base_weather.set_index('datetime_local', inplace=True)

#Concatenate the weather DataFrame to the base DataFrame 
base_df = pd.concat([base_df, base_weather], axis=1)

#Concatenate the incoming weight dataframe with the base dataframe
base_df = pd.concat([base_df, incoming_weight_1h], axis=1)
base_df = pd.concat([base_df, incoming_weight_5h], axis=1)
base_df = pd.concat([base_df, incoming_weight_10h], axis=1)
base_df = pd.concat([base_df, incoming_weight_23h], axis=1)

#Concatenate the pallet movement feature
base_df = pd.concat([base_df, pallet_move_5min], axis = 1)

# Drop all NaN values\n"
#base_df.dropna(subset=['demand_kW', 'Temperature'])


ValueError: Must specify a fill 'value' or 'method'.

Dummy drops things that are NA, maybe sth to look at

In [227]:
#dummy_df = base_df.dropna(subset=['demand_kW', 'Temperature', 'Relative Humidity']) 

dummy_df = base_df.copy()
dummy_df = dummy_df.reset_index()
dummy_df = dummy_df.drop(['hour'], axis=1)
dummy_df = dummy_df.drop(['datetime'], axis=1)
dummy_df = dummy_df.drop(['datetime_local'], axis=1)
dummy_df = dummy_df.drop(['datetime_UTC'], axis=1)
dummy_df

dummy_normalized_df = dummy_df.copy()
#Still drop demand_kW NaNs
dummy_normalized_df = dummy_normalized_df.dropna(subset=['demand_kW'], axis=0)

In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error

## Dataframe Column preprocessing functions

- Interpolate columns with missing values
- Normalizing columns (Min-max Normalization)
- One-hot encoding categorical columns

Quite important to do it in that order.

In [201]:
def normalize_column(df, columnname):
    """Function which returns a Dataframe where the given column is normalized through min-max normalization."""
    df[f'{columnname}_normalized'] = (df[columnname] - df[columnname].min()) / (df[columnname].max() - df[columnname].min())
    return df.drop([columnname], axis=1)

def add_one_hot_encoder(df, colname):
    """
    Function which returns a DataFrame where the given column has been removed and replaced by
    one-hot-encoding columns for each value in the original column.
    """
    onehot = pd.get_dummies(df[colname], prefix=colname)
    return df.drop(colname, axis=1).join(onehot)

def interpolate_column(df, colname):
    df[f'{colname}_interpolated'] = df[colname].interpolate(method='linear')
    return df.drop([colname], axis=1)

In [27]:
interpolate = ['Temperature', 'Relative Humidity']
to_normalize = ['Relative Humidity_interpolated', 'Temperature_interpolated']
add_one_hot_encoding = ['weekday', 'year', 'month']


for x in interpolate:
    dummy_normalized_df = interpolate_column(dummy_normalized_df, x)
for x in to_normalize:
    dummy_normalized_df = normalize_column(dummy_normalized_df, x)
for x in add_one_hot_encoding:
    dummy_normalized_df = add_one_hot_encoder(dummy_normalized_df, x)

In [28]:
dummy_normalized_df

,demand_kW,day,minute,Relative Humidity_interpolated_normalized,Temperature_interpolated_normalized,weekday_0.0,weekday_1.0,weekday_2.0,weekday_3.0,weekday_4.0,...,month_3.0,month_4.0,month_5.0,month_6.0,month_7.0,month_8.0,month_9.0,month_10.0,month_11.0,month_12.0
42,2064.101392,31.0,15.0,0.572375,0.442623,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
45,1874.002081,31.0,30.0,0.572375,0.442623,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
48,1988.168511,31.0,45.0,0.620183,0.426230,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
52,2022.795943,31.0,0.0,0.620183,0.426230,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
55,1986.981872,31.0,15.0,0.620183,0.426230,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
58,1921.142106,31.0,30.0,0.620183,0.426230,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
61,1956.130266,31.0,45.0,0.620183,0.426230,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
65,2039.546964,31.0,0.0,0.620183,0.426230,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
68,2100.701840,31.0,15.0,0.671746,0.409836,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
71,1901.260583,31.0,30.0,0.727504,0.393443,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Model Building
- 1 divide into train/test 
- 2 set target column
- 3 get accuracy

# 1. Train and Testset
Things to look at:
1. how to divide test/train/holdout set

In [30]:
from sklearn.model_selection import train_test_split

In [33]:
def test_model(model, data, debug=False):
    xtrain, ytrain, xtest, ytest = data
    if debug:
        print("Fitting model...")
    model.fit(xtrain, ytrain)
    if debug:
        print('Predicting...')
    acc = model.predict(xtest)
    if debug:
        print('Calculating mean absolute error...')
    return mean_absolute_error(list(ytest), acc)

In [31]:
#Define train, test sets\n",
train, test = train_test_split(dummy_normalized_df, shuffle=True)
X_train = train.copy().drop(['demand_kW'], axis=1)
Y_train = train['demand_kW']
X_test = test.copy().drop(['demand_kW'], axis=1)
Y_test = test['demand_kW']

# 3. Model Selection


In [32]:

from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor


## SVM


In [12]:
svm = SVR()
svm.fit(X_train, Y_train)
acc = svm.predict(X_test)
result = r2_score(list(Y_test), acc)
result

C:\Users\sonja\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


-6.817829768035466e-05

# Model Testing 

In [235]:
parameter_settings = {
    'n_estimators': [10, 20, 50],
    'criterion': ['squared_error', 'friedman_mse', 'poisson']#,
    #'max_depth': [2, 4, 6],
    #'min_samples_split': [2, 4, 8]
}
data = [X_train, Y_train, X_test, Y_test]
model = RandomForestRegressor

In [236]:
def model_optimizer(data, model, params):
    parameter_combinations = itertools.product(*params.values())
    results = {}
    for parameters in parameter_combinations:
        start = time.time()
        params_dict = dict(zip(params.keys(), parameters))
        print('Testing model with parameters: ' + str(params_dict))
        current_model = model(**params_dict)
        mae = test_model(current_model, data, debug = False)
        print(f'Mean Absolute Error = {mae}')
        modelname = str(params_dict)
        end = time.time()
        print(f'Time spent: {end-start} seconds. \n')
        results[modelname] = [mae, (end-start)]
    return results
        
test = model_optimizer(data, RandomForestRegressor, parameter_settings)

Testing model with parameters: {'n_estimators': 10, 'criterion': 'squared_error'}


KeyError: 'squared_error'